In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from replacers import RegexReplacer

from gensim.models.doc2vec import TaggedDocument
from gensim import utils

c:\users\laomd\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# remove HTML tag pattern
rc = re.compile(r"\<.*?\>")  
# Replacer class
replacer = RegexReplacer()
# Lemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [3]:
def constructTaggedDocuments(data):
    return [TaggedDocument (utils.to_unicode(row).split(), ['Text' + '_%s' % str(index)])
           for index, row in data.iteritems()]

In [4]:
# read data
train_text = pd.read_table("data/2/trainData.txt", encoding='utf-8', header=None)[0]
train_y = pd.read_table("data/2/trainLabel.txt", header=None)[0]

In [5]:
train_text[:5]

0    As long as there's been 3d technology, (1950's...
1    William Shakespeare's Merchant of Venice portr...
2    L'Auberge Espagnole is full of energy, and it'...
3    As I was reading through the comments, I was s...
4    The kind of B-movies from the 1950's that were...
Name: 0, dtype: object

In [6]:
train_y[:5]

0    0
1    1
2    1
3    1
4    0
Name: 0, dtype: int64

## 数据清洗

In [7]:
def SplitPhase(paragraph):
    """ split paragraph to sentences """
    PunktTokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()
    return PunktTokenizer.tokenize(paragraph)

def RemoveHTML(sentences):
    """ remove HTML tags """
    return [rc.sub('',sentence) for sentence in sentences]

def ReplaceAbbre(sentences):
    """ Replace abbreviation """
    return [replacer.replace(sentence) for sentence in sentences]

In [8]:
all_text = train_text.copy()

In [9]:
all_text[1]

'William Shakespeare\'s Merchant of Venice portrays 16th century Venice. Al Pacino plays Shylock, a Jewish loan shark who plots revenge on a Catholic that has looked down on him. The movie is a slow moving plot in the beginning that builds up throughout the two plus hours. The film gives a very good and believe appearance to it\'s characters, especially Pacino. When hearing that Pacino plays a Jew one might think that it would not work looking at Pacino\'s previous mobster type movie roles. Nonetheless it works very well, credit must be given to the costume designer\'s and director\'s of the film. The look of all the characters fits well with the time period the play takes place in. The costumes look like the Renaissance appearance one might envision to be.<br /><br />The film portrays a very anti-Semitic vibe. From the first minute to the last it is shown how the Catholic\'s try to take advantage of the Jews in every way they can, even to the point of keeping them locked away in "ghet

## 去掉HTML tag

In [10]:
no_html_text = all_text.apply(RemoveHTML)

In [23]:
no_html_text[1]

["William Shakespeare's Merchant of Venice portrays 16th century Venice.",
 'Al Pacino plays Shylock, a Jewish loan shark who plots revenge on a Catholic that has looked down on him.',
 'The movie is a slow moving plot in the beginning that builds up throughout the two plus hours.',
 "The film gives a very good and believe appearance to it's characters, especially Pacino.",
 "When hearing that Pacino plays a Jew one might think that it would not work looking at Pacino's previous mobster type movie roles.",
 "Nonetheless it works very well, credit must be given to the costume designer's and director's of the film.",
 'The look of all the characters fits well with the time period the play takes place in.',
 'The costumes look like the Renaissance appearance one might envision to be.The film portrays a very anti-Semitic vibe.',
 'From the first minute to the last it is shown how the Catholic\'s try to take advantage of the Jews in every way they can, even to the point of keeping them lock

## 替换缩写

In [ ]:
no_abbr_text = no_html_text.apply(ReplaceAbbre)

## 

In [10]:
sentences = constructTaggedDocuments(no_abbr_text)

In [14]:
sentences[1]

TaggedDocument(words=['william', "shakespeare's", 'merchant', 'venice', 'portrays', '16th', 'century', 'venice.', 'al', 'pacino', 'plays', 'shylock,', 'jewish', 'loan', 'shark', 'plots', 'revenge', 'catholic', 'looked', 'him.', 'movie', 'slow', 'moving', 'plot', 'beginning', 'builds', 'throughout', 'two', 'plus', 'hours.', 'film', 'gives', 'good', 'believe', 'appearance', 'characters,', 'especially', 'pacino.', 'hearing', 'pacino', 'plays', 'jew', 'one', 'might', 'think', 'would', 'work', 'looking', "pacino's", 'previous', 'mobster', 'type', 'movie', 'roles.', 'nonetheless', 'works', 'well,', 'credit', 'must', 'given', 'costume', "designer's", "director's", 'film.', 'look', 'characters', 'fits', 'well', 'time', 'period', 'play', 'takes', 'place', 'in.', 'costumes', 'look', 'like', 'renaissance', 'appearance', 'one', 'might', 'envision', 'be.<br', '/', '<br', '/', 'film', 'portrays', 'anti-semitic', 'vibe.', 'first', 'minute', 'last', 'shown', "catholic's", 'try', 'take', 'advantage', '

## 数据转vector

In [12]:
from gensim.models import Doc2Vec
from sklearn.linear_model import LogisticRegression     # 用逻辑回归做模型
from sklearn.model_selection import cross_val_score

In [12]:
VecType = Doc2Vec

In [13]:
VecType.__name__

'Doc2Vec'

##    Parameters
    def __init__(self, documents=None, corpus_file=None, dm_mean=None, dm=1, dbow_words=0, dm_concat=0,
                 dm_tag_count=1, docvecs=None, docvecs_mapfile=None, comment=None, trim_rule=None, callbacks=(),
                 **kwargs):
    ----------
    documents : iterable of list of :class:`~gensim.models.doc2vec.TaggedDocument`, optional
        Input corpus, can be simply a list of elements, but for larger corpora,consider an iterable that streams
        the documents directly from disk/network. If you don't supply `documents`, the model is
        left uninitialized -- use if you plan to initialize it in some other way.
    corpus_file : str, optional
        Path to a corpus file in :class:`~gensim.models.word2vec.LineSentence` format.
        You may use this argument instead of `sentences` to get performance boost. Only one of `sentences` or
        `corpus_file` arguments need to be passed (or none of them).
    dm : {1,0}, optional
        Defines the training algorithm. If `dm=1`, 'distributed memory' (PV-DM) is used.
        Otherwise, `distributed bag of words` (PV-DBOW) is employed.
    vector_size : int, optional
        Dimensionality of the feature vectors.
    window : int, optional
        The maximum distance between the current and predicted word within a sentence.
    alpha : float, optional
        The initial learning rate.
    min_alpha : float, optional
        Learning rate will linearly drop to `min_alpha` as training progresses.
    seed : int, optional
        Seed for the random number generator. Initial vectors for each word are seeded with a hash of
        the concatenation of word + `str(seed)`. Note that for a fully deterministically-reproducible run,
        you must also limit the model to a single worker thread (`workers=1`), to eliminate ordering jitter
        from OS thread scheduling.
        In Python 3, reproducibility between interpreter launches also requires use of the `PYTHONHASHSEED`
        environment variable to control hash randomization.
    min_count : int, optional
        Ignores all words with total frequency lower than this.
    max_vocab_size : int, optional
        Limits the RAM during vocabulary building; if there are more unique
        words than this, then prune the infrequent ones. Every 10 million word types need about 1GB of RAM.
        Set to `None` for no limit.
    sample : float, optional
        The threshold for configuring which higher-frequency words are randomly downsampled,
        useful range is (0, 1e-5).
    workers : int, optional
        Use these many worker threads to train the model (=faster training with multicore machines).
    epochs : int, optional
        Number of iterations (epochs) over the corpus.
    hs : {1,0}, optional
        If 1, hierarchical softmax will be used for model training.
        If set to 0, and `negative` is non-zero, negative sampling will be used.
    negative : int, optional
        If > 0, negative sampling will be used, the int for negative specifies how many "noise words"
        should be drawn (usually between 5-20).
        If set to 0, no negative sampling is used.
    ns_exponent : float, optional
        The exponent used to shape the negative sampling distribution. A value of 1.0 samples exactly in proportion
        to the frequencies, 0.0 samples all words equally, while a negative value samples low-frequency words more
        than high-frequency words. The popular default value of 0.75 was chosen by the original Word2Vec paper.
        More recently, in https://arxiv.org/abs/1804.04212, Caselles-Dupré, Lesaint, & Royo-Letelier suggest that
        other values may perform better for recommendation applications.
    dm_mean : {1,0}, optional
        If 0 , use the sum of the context word vectors. If 1, use the mean.
        Only applies when `dm` is used in non-concatenative mode.
    dm_concat : {1,0}, optional
        If 1, use concatenation of context vectors rather than sum/average;
        Note concatenation results in a much-larger model, as the input
        is no longer the size of one (sampled or arithmetically combined) word vector, but the
        size of the tag(s) and all words in the context strung together.
    dm_tag_count : int, optional
        Expected constant number of document tags per document, when using
        dm_concat mode.
    dbow_words : {1,0}, optional
        If set to 1 trains word-vectors (in skip-gram fashion) simultaneous with DBOW
        doc-vector training; If 0, only trains doc-vectors (faster).
    trim_rule : function, optional
        Vocabulary trimming rule, specifies whether certain words should remain in the vocabulary,
        be trimmed away, or handled using the default (discard if word count < min_count).
        Can be None (min_count will be used, look to :func:`~gensim.utils.keep_vocab_item`),
        or a callable that accepts parameters (word, count, min_count) and returns either
        :attr:`gensim.utils.RULE_DISCARD`, :attr:`gensim.utils.RULE_KEEP` or :attr:`gensim.utils.RULE_DEFAULT`.
        The rule, if given, is only used to prune vocabulary during current method call and is not stored as part
        of the model.

        The input parameters are of the following types:
            * `word` (str) - the word we are examining
            * `count` (int) - the word's frequency count in the corpus
            * `min_count` (int) - the minimum count threshold.

    callbacks : :obj: `list` of :obj: `~gensim.models.callbacks.CallbackAny2Vec`, optional
        List of callbacks that need to be executed/run at specific stages during training.


In [13]:
filename='trained_vecs/2/trainData_clean.vec'
text_model = Doc2Vec(min_count=1, window=5, 
                     vector_size=150, 
                     negative=25, workers=4, 
                     epochs=25, alpha=0.05, seed=4)
text_model.build_vocab(sentences)
text_model.train(sentences, total_examples=text_model.corpus_count, epochs=text_model.epochs)

KeyboardInterrupt: 

In [22]:
text_model.save(filename)
# text_model = Doc2Vec.load(filename)

In [23]:
data = np.array([text_model.docvecs['Text_'+str(i)] for i in range(len(train_text))])

In [24]:
data[:5]

array([[ 8.97976339e-01, -7.52540231e-01,  2.04513431e+00,
        -8.55905533e-01, -2.29428172e+00, -1.99305296e+00,
        -1.53948152e+00,  4.97840822e-01, -7.53712535e-01,
         1.28665245e+00, -8.77376676e-01,  7.33760953e-01,
        -1.17904317e+00,  4.22223330e-01,  8.00953448e-01,
        -2.71314240e+00, -2.69650400e-01,  1.90351105e+00,
        -1.12113571e+00, -5.28665245e-01, -4.82490778e-01,
        -2.89393616e+00, -3.64178754e-02,  1.03160334e+00,
         1.20031405e-02, -1.09143043e+00, -4.24810499e-01,
         1.24755955e+00,  1.40458322e+00,  7.50815094e-01,
         6.11275546e-02, -7.51111209e-01, -7.95800745e-01,
         1.43203139e+00,  5.22482991e-01, -4.60974169e+00,
        -1.20615780e+00, -1.52807903e+00, -1.48960841e+00,
         1.95232081e+00, -2.35811925e+00,  2.04465318e+00,
        -8.03631186e-01, -7.55493879e-01,  5.69580138e-01,
         1.00461555e+00,  3.68598342e-01,  1.31550372e+00,
        -1.13665140e+00,  8.97321045e-01,  4.59804088e-0

In [25]:
model = LogisticRegression()
cross_val_score(model, data, train_y, cv=5, scoring='accuracy').mean()   # window=5, dm=1(distributed memory), min_count=1

c:\users\laomd\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\laomd\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\laomd\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\laomd\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\laomd\appdata\local\programs\python\python36\lib\si

0.8710001887659224

In [33]:
from importlib import reload
from logistic_model import logistic_regression, base
reload(base)
reload(logistic_regression)
from logistic_model.logistic_regression import SimpleLogisticRegression

In [32]:
clf = SimpleLogisticRegression()
clf.fit(data, train_y)

In [34]:
cross_val_score(SimpleLogisticRegression(), data, train_y, cv=5, scoring='accuracy').mean()   # window=5, dm=1(distributed memory), min_count=1

0.8714583333333333